_Carlos Cesar Caballero Díaz_

https://github.com/cccaballero/miav-reconocimiento-patrones-agrupamiento

Se desea reducir el tamaño de imágenes digitales en formato PNG reduciendo el número de colores que se encuentran en una imagen a solo los más comunes en dicha imagen. Se desean conformar grupos de píxeles a partir de la similitud en sus canales de colores, los cuales serán convertidos a un color único para, de dicha forma, reducir el tamaño total de la imagen.

A partir del objeto Pixel se determinan los siguientes rasgos:

| Rasgo | Nombre | Dominio | Descripción |
| --- | --- | --- | --- |
| R1 | Rojo | {0-255} | Valor del canal rojo |
| R2 | Verde | {0-255} | Valor del canal verde |
| R3 | Azul | {0-255} | Valor del canal azul |


Se obtienen los datos de los píxeles a partir de la imagen de referencia

In [16]:
from PIL import Image
import random
import pandas as pd

im = Image.open('image.png')
pix_list = list(im.getdata())

Se selección una parte de la base de datos para conformar la matriz inicial.

In [17]:
random.seed(7)
pix_sample = random.sample(pix_list, 5)
learn_matrix = pd.DataFrame(pix_sample, columns=["R1", "R2", "R3"])
learn_matrix

,R1,R2,R3
0,12,17,13
1,34,55,22
2,151,140,114
3,41,147,206
4,79,142,198


Como función de semejanza entre objetos se utilizará "CIE 2000" para el cálculo de  
"Delta E" para determinar la diferencia entre colores (https://onlinelibrary.wiley.com/doi/abs/10.1002/col.1049). Dicho algoritmo devuelve valores entre 0 y 206.8043 siendo 0 dos colores iguales, y 206.8043 la máxima diferencia posible. El resultado de dicho algoritmo se lleva a la escala de 0...1 y se invierte para obtener a 1 como valor de igualdad de la siguiente forma:

$\sigma_i(O_0,O_t) = \text{Valor de diferencia Delta E para ambos objetos}$

$$\beta(O_0,O_t) = \frac{(\frac{\sigma_i(O_0,O_t) * 100}{206.8043})}{100}$$


In [18]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

def compare_objects(o1, o2):
    color_o1 = convert_color(sRGBColor(*o1), LabColor)
    color_o2 = convert_color(sRGBColor(*o2), LabColor)
    return 1 - ((delta_e_cie2000(color_o1, color_o2)*100/206.8043) / 100)

Se obtiene la matriz de semejanza:

In [19]:
import numpy
msa = numpy.zeros(shape=(len(learn_matrix),len(learn_matrix)))
pd.DataFrame(msa)
for i, o1 in learn_matrix.iterrows():
    for j in range(i,len(learn_matrix)):
        cmp = compare_objects(o1,learn_matrix.values[j])
        msa[i,j] = msa[j,i] = cmp
pd.DataFrame(msa)

,0,1,2,3,4
0,1.000000,0.685449,0.481213,0.436606,0.436613
1,0.685449,1.000000,0.655194,0.377954,0.380378
2,0.481213,0.655194,1.000000,0.448235,0.415261
3,0.436606,0.377954,0.448235,1.000000,0.953166
4,0.436613,0.380378,0.415261,0.953166,1.000000


Se define un umbral de semejanza a criterio de experto de:

$\beta_0 = 0.9$

Se procede a realizar el agrupamiento segun el criterio $\beta_0$ - conexa

In [20]:
# comprueba si "obj" pertenece a algun grupo de "groups"
def belongs_to_group(obj, groups):
    for group in groups:
        for element in group:
            if obj == element:
                return True
    return False

beta = 0.9
groups = []
for i, row in enumerate(msa):
    group = []
    if belongs_to_group(i, groups):
        continue
    else:
        group.append(i)
    for j, element in enumerate(row):
        if i != j and element >= beta:
            group.append(j)
    groups.append(group)
groups

[[0], [1], [2], [3, 4]]